In [11]:
import json
import math
import numpy as np
import pandas as pd
import pickle as pickle
import datetime
from datetime import date

In [12]:
data_name = 'harris'
data_root_path = '/data/zhimengj/Human_Mob/Harris/'
data_save_path = '/data/zhimengj/dataset/Harris/'
Grid_PATH = data_root_path + 'Grid_Map/Harris_010.p'

### save nodes features/labels
Nodes_PATH = data_root_path + 'Node_Features/Harris_all_labels.csv'
OUT_NODE_FEAT = data_save_path + '{}_node.npy'.format(data_name)
OUT_NODE_SFEAT = data_save_path + '{}_Snode.p'.format(data_name)
OUT_NODE_label = data_save_path + '{}_Ynode.npy'.format(data_name)
OUT_NODE_label2 = data_save_path + '{}_Ynode2.npy'.format(data_name)
### save temporal links
time_duration = 1
OUT_DF = data_save_path + '{}_edge_{}.csv'.format(data_name, time_duration)
OUT_FEAT = data_save_path + '{}_edge_{}.npy'.format(data_name, time_duration)

In [13]:
with open(Grid_PATH, "rb") as f:
    grid = pickle.load(f) ## remove column 0, 5, 9, 10
index_name = grid.index.tolist()
index_name

[(1, 64),
 (1, 65),
 (2, 62),
 (2, 63),
 (2, 64),
 (3, 59),
 (3, 60),
 (3, 61),
 (3, 62),
 (3, 63),
 (3, 64),
 (4, 56),
 (4, 57),
 (4, 58),
 (4, 59),
 (4, 60),
 (4, 61),
 (4, 62),
 (4, 63),
 (5, 53),
 (5, 54),
 (5, 55),
 (5, 56),
 (5, 57),
 (5, 58),
 (5, 59),
 (5, 60),
 (5, 61),
 (5, 62),
 (5, 63),
 (6, 50),
 (6, 51),
 (6, 52),
 (6, 53),
 (6, 54),
 (6, 55),
 (6, 56),
 (6, 57),
 (6, 58),
 (6, 59),
 (6, 60),
 (6, 61),
 (6, 62),
 (7, 48),
 (7, 49),
 (7, 50),
 (7, 51),
 (7, 52),
 (7, 53),
 (7, 54),
 (7, 55),
 (7, 56),
 (7, 57),
 (7, 58),
 (7, 59),
 (7, 60),
 (7, 61),
 (7, 62),
 (8, 45),
 (8, 46),
 (8, 47),
 (8, 48),
 (8, 49),
 (8, 50),
 (8, 51),
 (8, 52),
 (8, 53),
 (8, 54),
 (8, 55),
 (8, 56),
 (8, 57),
 (8, 58),
 (8, 59),
 (8, 60),
 (8, 61),
 (8, 62),
 (9, 42),
 (9, 43),
 (9, 44),
 (9, 45),
 (9, 46),
 (9, 47),
 (9, 48),
 (9, 49),
 (9, 50),
 (9, 51),
 (9, 52),
 (9, 53),
 (9, 54),
 (9, 55),
 (9, 56),
 (9, 57),
 (9, 58),
 (9, 59),
 (9, 60),
 (9, 61),
 (9, 62),
 (10, 39),
 (10, 40),
 (10, 41

In [14]:
Nodes = pd.read_csv(Nodes_PATH)
# index_name = Nodes.index.tolist()
Nodes = Nodes.drop(columns=['box', 'Aux_Stops', 'Aux_Visits', 'Aux_Time'])


In [15]:
Sens_features = Nodes.loc[:,['% White', '% Black']]
Sen_index = np.invert(pd.isna(Sens_features)).to_numpy()
Sen_index = Sen_index[:,0] * Sen_index[:,1]
# Sens_features2 = Sens_features[Sen_index].to_numpy()
Sens_features = Sens_features.to_numpy()
print(f'Sen_index={Sen_index.shape}')
print(f'Sens_features={Sens_features.shape}')
# Sens_features_dict = {"sfeature": Sens_features, "index": Sen_index}
Sens_features_filter = Sens_features[Sen_index, 0] / 100 ### white people ratio
pickle.dump(Sens_features_filter, open(OUT_NODE_SFEAT, "wb") )
Sen_index

Sen_index=(4287,)
Sens_features=(4287, 2)


array([ True, False,  True, ..., False, False, False])

In [16]:
index_name_filter = []
for i in range(len(Sen_index)):
    if Sen_index[i]:
        index_name_filter.append(index_name[i])
len(index_name_filter)

4204

In [17]:
### save Nodes features
Nodes_features_filter = Nodes.iloc[Sen_index,0:36].to_numpy()
np.save(OUT_NODE_FEAT, Nodes_features_filter)
Nodes_features_filter.shape

(4204, 36)

In [18]:
### save Nodes labels
Nodes_labels = Nodes['Per capita income'].to_numpy()
Label_index = np.invert(pd.isna(Nodes_labels))
Nodes_labels = (Nodes_labels > np.median(Nodes_labels[Label_index])).astype(int)
Nodes_labels_filter = Nodes_labels[Sen_index]
np.save(OUT_NODE_label, Nodes_labels_filter)
Nodes_labels_filter

array([0, 0, 0, ..., 1, 1, 1])

In [19]:
### save Nodes labels 2
Nodes_labels = Nodes['Per capita income'].to_numpy()
Label_index = np.invert(pd.isna(Nodes_labels))
# Nodes_labels = (Nodes_labels > np.median(Nodes_labels[Label_index])).astype(int)
Nodes_labels_filter2 = Nodes_labels[Sen_index]
Nodes_labels_filter2 = Nodes_labels_filter2 / np.max(Nodes_labels_filter2)
np.save(OUT_NODE_label2, Nodes_labels_filter2)
Nodes_labels_filter2

array([0.16591755, 0.16591755, 0.16591755, ..., 0.20307876, 0.19884808,
       0.20307876])

In [20]:
start_date = '2017-09-01'
date_1 = datetime.datetime.strptime(start_date, "%Y-%m-%d")

count = 0 
sum = 0
ratio = 0.1
for time_index in range(time_duration):
    end_date = date_1 + datetime.timedelta(days=time_index)
    end_date = end_date.strftime("%Y-%m-%d")
    Links_PATH = data_root_path + 'Links/{}.csv'.format(end_date)
    Links = pd.read_csv(Links_PATH)
    count += Links.shape[0]
    num_link = math.floor(ratio*Links.shape[0])
    sum += np.sum(Links.iloc[:num_link, 5])

count_th = sum / count
Links

,Unnamed: 0,Origin_ID,Destination_ID,Date,Time,Count
0,0,"(46, 64)","(46, 64)",2017-09-01,12,38
1,1,"(44, 39)","(44, 39)",2017-09-01,20,8
2,2,"(26, 47)","(51, 23)",2017-09-01,8,1
3,3,"(51, 23)","(51, 23)",2017-09-01,12,35
4,4,"(51, 23)","(34, 44)",2017-09-01,20,1
...,...,...,...,...,...,...
355497,355497,"(42, 37)","(39, 45)",2017-09-01,8,1
355498,355498,"(38, 55)","(36, 51)",2017-09-01,8,1
355499,355499,"(36, 51)","(37, 54)",2017-09-01,8,1
355500,355500,"(17, 27)","(18, 26)",2017-09-01,8,1


In [21]:
u_list, i_list, ts_list, label_list = [], [], [], []
idx_list = []
idx = 0
for time_index in range(time_duration):
    end_date = date_1 + datetime.timedelta(days=time_index)
    end_date = end_date.strftime("%Y-%m-%d")
    Links_PATH = data_root_path + 'Links/{}.csv'.format(end_date)
    Links = pd.read_csv(Links_PATH)
    num_link = math.floor(ratio*Links.shape[0])
    for index in range(num_link):
        src = eval(Links.iloc[index, 1])
        des = eval(Links.iloc[index, 2])
        if src in index_name_filter and des in index_name_filter and src != des:
            u = index_name_filter.index(src)
            i = index_name_filter.index(des)
            if Sen_index[u] and Sen_index[i]:
                ts = (time_index * 24 + Links.iloc[index, 4]) / 4
                label = int(Links.iloc[index, 5] > count_th)
                idx += 1

                u_list.append(u)
                i_list.append(i)
                ts_list.append(ts)
                label_list.append(label)
                idx_list.append(idx)

E_feat = pd.DataFrame({'u': u_list, 
                    'i':i_list, 
                    'ts':ts_list, 
                    'label':label_list, 
                    'idx':idx_list})
E_feat.to_csv(OUT_DF)
E_feat

,u,i,ts,label,idx
0,610,1774,2.0,1,1
1,1774,939,5.0,1,2
2,939,683,5.0,1,3
3,1232,1281,5.0,1,4
4,1281,1495,5.0,1,5
...,...,...,...,...,...
19941,134,309,3.0,1,19942
19942,309,134,4.0,1,19943
19943,2865,3060,4.0,1,19944
19944,1267,1117,4.0,1,19945


In [22]:
feat = np.random.uniform(0, 1, size=(E_feat.shape[0], 36))
np.save(OUT_FEAT, feat)

In [23]:
feat = np.load(OUT_FEAT)
feat.shape

(19946, 36)

In [24]:
np.mean(E_feat['label'])

1.0